# Process model outputs 

In [2]:
import os 
import sys
import dill as pickle
import itertools as it
import numpy as np
import pandas as pd
import imageio.v2 as imageio 
from tqdm.notebook import tqdm
from pathlib import Path
import shutil


project_dir = os.path.abspath('../../')
data_dir = os.path.join(project_dir, 'data')
model_dir = os.path.join(data_dir, 'models')
pickle_dir = os.path.join(model_dir, 'pickles')
records_dir = os.path.join(model_dir, 'records')

sys.path.append(os.path.join(project_dir, 'gym-cooking', 'gym_cooking'))
# import recipe_planner

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

## Process model data

In [2]:
intention_agent = ['1agent', '2agent', 'either_agents']
intention_recipe = ['onion', 'tomato', 'either_recipe']

model_agent = ['1agent', '2agent']
model_recipe = ['onion', 'tomato']

n_seed = 20

In [3]:
df = []
for ia, ir, ma, mr in it.product(intention_agent, intention_recipe, model_agent, model_recipe):
    for seed_num in range(1, n_seed + 1):
        fname = os.path.join(pickle_dir, f'{ia}-{ir}', f'{ma}-{mr}-seed{seed_num}.pkl')
        if os.path.exists(fname):
            with open(fname, 'rb') as f:
                data = pickle.load(f)

                agents = list(data['actions'].keys())
                df.append({
                    'intended_num_agents': ia,
                    'intended_recipe': ir,
                    'intention': f'{ia}-{ir}',
                    'model_num_agents': ma,
                    'model_recipe': mr,
                    'model_run': f'{ma}-{mr}',
                    'design_congruent': 'congruent' if (ia == ma) & (ir == mr) else 'incongruent',
                    'seed': seed_num,
                    'timesteps': len(data['actions']['agent-1']),
                    'agent_pauses': sum([a == (0,0) for agent in agents for a in data['actions'][agent]]),
                    'agent_collisions': len(data['collisions']),
                    'was_successful': data['was_successful']
                })

model_df = pd.DataFrame(df)
model_df.to_csv(os.path.join(model_dir, 'model_results.csv'), index=False)

## Render gifs


In [4]:
def create_gifs_from_records(
    model_dir: str, 
    framerate: float = 3.0, 
    overwrite: bool = False
):
    """
    Walk `model_dir/records/<intent>/<level>/seed=*` and, for each seed,
    bundle all .pngs into `model_dir/gifs/<intent>/<level>/seed=<N>.gif`.

    Shows two nested progress bars:
      - Outer: over all intent/level directories (unit="dir")
      - Inner: over 20 seeds inside each directory (unit="seed")

    Parameters
    ----------
    model_dir : str
        Path to the top‐level model folder (e.g. "data/models").
    framerate : float
        Frames per second in each output GIF.
    overwrite : bool
        If True, delete any existing GIFs for that intent/level before regenerating.
        If False, skip seeds where the target GIF already exists.
    """
    records_root = Path(model_dir) / "records"
    gifs_root    = Path(model_dir) / "gifs"

    if not records_root.exists():
        raise FileNotFoundError(f"records_root not found: {records_root}")

    # 1) Build a flat list of all (intent_dir, level_dir) pairs
    all_dirs = []
    for intent_dir in sorted(records_root.iterdir()):
        if not intent_dir.is_dir():
            continue
        for level_dir in sorted(intent_dir.iterdir()):
            if not level_dir.is_dir():
                continue
            # Only include if it has at least one seed=* subfolder
            seed_dirs = [d for d in level_dir.iterdir() if d.is_dir() and d.name.startswith("seed=")]
            if seed_dirs:
                all_dirs.append((intent_dir, level_dir))

    # 2) Outer tqdm over all intent/level directories
    outer = tqdm(
        all_dirs,
        desc="Directory",
        unit="dir",
    )

    for intent_dir, level_dir in outer:
        rel = f"{intent_dir.name}/{level_dir.name}"
        # Update the outer description to show the current rel path
        outer.set_description(f"Directory {rel}")

        # prepare destination folder for these GIFs (and optionally wipe)
        gif_level_dir = gifs_root / intent_dir.name / level_dir.name
        if overwrite and gif_level_dir.exists():
            shutil.rmtree(gif_level_dir)
        gif_level_dir.mkdir(parents=True, exist_ok=True)

        # collect all seed subfolders under this level
        seed_dirs = sorted(
            [d for d in level_dir.iterdir() if d.is_dir() and d.name.startswith("seed=")]
        )
        if not seed_dirs:
            continue

        # 3) Inner tqdm over all seed folders in this level
        inner = tqdm(
            seed_dirs,
            desc="Seed",
            unit="seed",
            position=1,
            leave=False,
        )
        for seed_dir in inner:
            # Extract numeric seed (e.g. "seed=2" → "2")
            seed_num = seed_dir.name.split("=", 1)[1]
            inner.set_description(f"Seed {seed_num}")

            gif_path = gif_level_dir / f"{seed_dir.name}.gif"
            if gif_path.exists() and not overwrite:
                continue

            # gather all pngs in this seed folder
            png_files = sorted(seed_dir.glob("*.png"))
            if not png_files:
                continue

            # write the GIF
            with imageio.get_writer(str(gif_path), mode="I", fps=framerate) as writer:
                for png in png_files:
                    frame = imageio.imread(str(png))
                    writer.append_data(frame)

        inner.close()

    outer.close()
    
    out = shutil.make_archive(
            base_name=Path(gifs_root, 'model_run_gifs'), 
            format="zip", 
            root_dir=str(gifs_root),
            base_dir=".")
    
    print(f'Saved to {out}')


In [5]:
reallyRun = True
if reallyRun:
    create_gifs_from_records(model_dir, framerate=3, overwrite=True)

Directory:   0%|          | 0/36 [00:00<?, ?dir/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

...

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Seed:   0%|          | 0/20 [00:00<?, ?seed/s]

Saved to /Users/justyang/Code/design-inference/data/models/gifs/model_run_gifs.zip
